In [ ]:
# importa macro %%cu
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

# carrega plugin
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qqj_y3j2
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qqj_y3j2
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=9a9a92c92fce77da33e82dd724e41031b603fd164b9f0e3cc5cab5f22c778d73
  Stored in directory: /tmp/pip-ephem-wheel-cache-5mznbu7x/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%cd /usr/local
%cd cuda-10.0
%cd samples
%cd 1_Utilities
%cd deviceQuery
!ls
!make
!./deviceQuery

/usr/local
/usr/local/cuda-10.0
/usr/local/cuda-10.0/samples
/usr/local/cuda-10.0/samples/1_Utilities
/usr/local/cuda-10.0/samples/1_Utilities/deviceQuery
deviceQuery.cpp  Makefile  NsightEclipse.xml  readme.txt
/usr/local/cuda-10.0/bin/nvcc -ccbin g++ -I../../common/inc  -m64    -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o deviceQuery.o -c deviceQuery.cpp
/usr/local/cuda-10.0/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch

Universidade Federal de São Carlos\
Departamento de Computação\
Prof. Hélio Crestana Guardia\
Programação Paralela e Distribuída\

**Autores:** Vinicius Henrique dos Santos Carvalho 743602 e João Augusto Leite 743551 \

**Programa :** Multiplicação de matrizes utilizando CUDA\
**Estratégias:** Nesse código, foi abordada a técnica de calcular cada célula
em uma thread. Desse modo, a arquitetura tem 1 grid (1,1,1) e bloco contendo
uma thread por célula, ou seja (n,n,1).
As matrizes foram armazenadas linearmente.\
Para fazer os cálculos no Kernel, primeiro obtém-se a posição da célula
que será calculada naquela thread. Para isso, é obtido o número da linha multiplicando o ID do bloco no eixo Y pelo número de elementos de cada linha, possibilitando acessar o primeiro elemento dela e em seguida soma-se o ID da
thread no eixo Y. Para achar a coluna, utiliza-se o mesmo cálculo, porém
subtituindo os dados de Y por X.
Com a posição da célula que será cálculada, é feito um For para percorrer
toda a linha da primeira matriz e toda a coluna da segunda, mutiplicando
cada um desses elementos e somando, até o final da iteração, que é o limite
da matriz.
 

In [ ]:
# %%gpu
%%cu
#include <stdio.h>
#include <stdlib.h>

#define DIMENSAO 32
#define MAX_VALUE 9

//Função para iniciar a matriz com valores aleatórios até o MAX_VALUE definido
void init_matrix(int width, int height,int *m)
{
    int i, j;
    for(i=0; i<width; i++){
        for(j=0; j<height; j++){
            m[i*width+j] = rand() % MAX_VALUE;
            }
    }
}

void print_matrix(int width, int height, int *m)
{
    int i, j;

    for(i=0; i<width; i++)
    {
        printf("\n%d: ", i);
        for(j=0; j<height; j++)
            printf("%d ", m[i*width+j]);
    }
}


//Kernel CUDA que faz a mutiplicação
__global__ void MultiplicaMatriz(int *a, int *b, int *c, int n)
{
  //Cálculo para obter a posição do elemento da matriz no vetor
  //(blockIdx.x,blockIdx.y) são as coordenadas da thread no bloco
  //que também representa a célula que será calculada naquela thread
	int lin = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
 
	int soma = 0;
	for (int k = 0; k < n; k++) {
		soma += a[lin*n + k] * b[k * n + col];
	}
	c[lin*n + col] = soma;
}

int main(){
    
    int n = DIMENSAO;
    int tamanho = n*n;
    int *g_m1, *g_m2, *g_r;
    int m1[tamanho], m2[tamanho], resultado[tamanho];

    //Aloca memória das matrizes na GPU
	  cudaMalloc(&g_m1, sizeof(int)*tamanho);
	  cudaMalloc(&g_m2, sizeof(int)*tamanho);
	  cudaMalloc(&g_r, sizeof(int)*tamanho);

    printf("\nMatriz 1:\n");
    init_matrix(n, n, m1);
    print_matrix(n, n, m1);
    
    printf("\nMatriz 2:\n");
    init_matrix(n, n, m2);
    print_matrix(n, n, m2);
    
    //Copia as matrizes do host para a GPU
    cudaMemcpy(g_m1, m1, sizeof(int)*tamanho, cudaMemcpyHostToDevice);
    cudaMemcpy(g_m2, m2, sizeof(int)*tamanho, cudaMemcpyHostToDevice);

    //Define a dimensão do bloco, nesse caso NxN threads
    dim3 threadsPerBlock(n, n, 1);
    //Define a dimensão da grade, nesse caso, somente uma será usada
	  dim3 blocksPerGrid(1, 1, 1);

    //Inicia o Kernel passando as dimensões dos blocos e grid, as duas matrizes 
    //que serão mutiplicadas, a matriz que armazenará o resultado e o 
    //número elementos por linha/coluna
    MultiplicaMatriz<<<dimGrid, threadsPerBlock>>>(g_m1, g_m2, g_r, n);

    //Verifica se houve algum erro ao ativar o Kernel
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
      printf("CUDA Error: %s\n",cudaGetErrorString(err));
    }

    //Função para o Host aguardar a execução de todas as threads
    cudaDeviceSynchronize();

    //Copia a matriz resultante da GPU para o HOST
    cudaMemcpy(resultado, g_r, sizeof(int)*tamanho, cudaMemcpyDeviceToHost);

    printf("\nMatriz resultado:\n");
    print_matrix(n, n, resultado);

    //DLibera a memória alocada na GPU e a reseta
    cudaFree(g_m1);
    cudaFree(g_m2);
    cudaFree(g_r);

    cudaDeviceReset();

    return 0;
}


Matriz 1:

0: 1 7 0 7 5 7 1 3 6 1 5 4 5 7 5 4 6 0 7 1 8 8 6 6 8 8 8 4 1 1 5 0 
1: 0 3 5 3 1 7 4 7 6 0 0 2 5 4 5 2 2 3 2 1 1 8 8 0 5 5 4 4 6 0 5 6 
2: 2 8 7 3 4 2 0 0 0 0 2 6 2 5 6 5 7 6 6 8 5 3 6 2 8 1 6 6 8 0 1 1 
3: 7 0 3 2 0 1 2 1 8 3 5 2 6 0 7 2 7 2 8 1 6 5 1 5 4 6 0 4 6 2 3 2 
4: 0 4 3 7 5 3 6 5 4 2 5 2 1 3 2 8 3 2 0 0 7 2 4 3 6 2 5 1 2 6 4 2 
5: 2 7 8 5 1 5 1 4 8 4 6 7 5 8 6 0 8 4 0 7 4 2 8 1 5 2 3 7 8 7 8 1 
6: 3 7 5 2 3 6 6 0 2 1 7 7 8 2 5 7 7 4 3 0 6 0 2 0 2 3 6 2 8 5 1 2 
7: 1 6 2 2 2 7 3 2 6 8 7 3 0 4 2 5 8 3 5 3 4 5 4 4 8 1 6 7 4 6 7 5 
8: 3 8 7 3 6 8 5 1 6 2 5 6 6 5 0 3 8 5 6 1 2 8 6 8 0 1 7 2 7 3 7 2 
9: 2 4 3 6 1 0 8 7 2 2 2 6 7 2 0 6 6 4 6 6 4 3 5 2 4 1 3 3 5 8 3 5 
10: 3 6 1 3 4 7 8 4 0 2 1 7 2 8 2 6 2 8 3 4 0 7 4 5 8 7 6 2 7 0 8 8 
11: 4 0 2 7 7 2 2 5 2 2 3 2 8 3 0 1 3 1 5 1 8 8 6 8 6 1 1 4 8 7 4 4 
12: 5 4 2 1 4 2 6 6 4 7 7 2 2 7 3 3 6 7 4 6 6 0 3 3 1 2 6 1 1 8 3 6 
13: 1 3 8 6 5 3 3 8 2 1 1 4 6 2 5 4 0 0 8 4 7 2 6 0 4 3 8 3 0 2 8 1 
14: 5 5 7 8 8 0 7 1 8 8 3 6 2 8 